# Image-to-Image Translation : *pix2pix*

## Download scripts & datasets

In [1]:
# !git clone https://github.com/whieya/NPEX/GAN_projects
# !bash ./datasets/download_pix2pix_dataset.sh edges2handbags

# download dataset
!mkdir ./datasets
!wget -N http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz -O ./datasets/facades.tar.gz
!mkdir -p ./datasets/facades/
!tar -zxvf ./datasets/facades.tar.gz -C ./datasets/


for details.

--2020-08-13 17:30:27--  http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30168306 (29M) [application/x-gzip]
Saving to: ‘./datasets/facades.tar.gz’

./datasets/facades. 100%[===================>]  28.77M  1.64MB/s    in 30s     

2020-08-13 17:30:58 (968 KB/s) - ‘./datasets/facades.tar.gz’ saved [30168306/30168306]

facades/
facades/test/
facades/test/27.jpg
facades/test/5.jpg
facades/test/72.jpg
facades/test/1.jpg
facades/test/10.jpg
facades/test/100.jpg
facades/test/101.jpg
facades/test/102.jpg
facades/test/103.jpg
facades/test/104.jpg
facades/test/105.jpg
facades/test/106.jpg
facades/test/11.jpg
facades/test/12.jpg
facades/test/13.jpg
facades/test/14.jpg
facades/test/15.jpg
facades/test/16.jpg
facades/test/17.j

### Prepare DataLoader for MNIST dataset

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# fix manual seed.
torch.manual_seed(1234)

# set batch size.
BATCH_SIZE = 8
NUM_EXAMPLES = 8

In [3]:
import os
from torch.utils.data import Dataset
import PIL

# Custom Dataset 
class CustomFacades(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.transform = transform       
        self.train = train
        self.dir = 'train' if train else 'test'
        self.len_train = 400
        self.len_test = 106

    def __len__(self):
        if self.train:
            return self.len_train
        else:
            return self.len_test

    def __getitem__(self, idx):
        data_idx = idx if self.train else idx + self.len_train
        X = PIL.Image.open(os.path.join(self.root, self.dir, '{}.jpg'.format(idx+1)))
        if self.transform is not None:
            X = self.transform(X)
        return X[..., 256:], X[..., :256]

In [4]:
# prepare dataloader.
tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

# train loader for training GAN
train_dataset = CustomFacades(root='./datasets/facades', train=True, 
        transform=tf)
train_loader = DataLoader(dataset=train_dataset, num_workers=8, batch_size=BATCH_SIZE, shuffle=True)

# test loader for examining test samples
test_dataset = CustomFacades(root='./datasets/facades', train=False, transform=tf)
test_loader = DataLoader(dataset=test_dataset, num_workers=8, batch_size=BATCH_SIZE, shuffle=False)

### Define GANs Models

#### Define Generator

In [5]:
import torch.nn as nn

# Unet Generator for pix2pix model.
# https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix 
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer
        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

import functools
class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.
        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)


#### Define Discriminator

In [6]:
# Unet Discriminator for pix2pix model.
# https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix 
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

#### Prepare GAN model and initialize the weights

In [7]:
#weight initialization function. 
def weights_init(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Linear):
            # m.weight.data.normal_(1.0, 0.2)
            nn.init.xavier_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.normal_(1.0, 0.2)
            m.bias.data.zero_()

# define GAN model.
G = UnetGenerator(input_nc=3, output_nc=3, num_downs=8).cuda()
D = NLayerDiscriminator(input_nc=3+3, ndf=64, n_layers=3).cuda()

# weight initialization 
G.apply(weights_init)
D.apply(weights_init)

NLayerDiscriminator(
  (model): Sequential(
    (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

#### Start training GAN

In [8]:
# install tensorboardx to use tensorboard.
%pip install tensorboardx

from tensorboardX import SummaryWriter
from torchvision.utils import make_grid
import time

# Hyper-parameters. 
# ====== You don't need to change here ===== #
EPOCHS = 200
Z_DIM = 64
LAMBDA_L1 = 100
# ========================================== #

# logger for tensorboard.
logger = SummaryWriter()

# GT labels for calculating binary cross entropy loss. 
real_label = torch.ones(size=(BATCH_SIZE,1)).cuda()
fake_label = torch.zeros(size=(BATCH_SIZE,1)).cuda()

# criterion for binary cross entropy loss
BCE_criterion = torch.nn.BCELoss()

# define optimizer. Here we use Adam optimizer. 
optimizer_G = torch.optim.Adam(G.parameters(), lr=2e-4, betas=(0.5,0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=2e-4, betas=(0.5,0.999))

iterations = 0

for epoch in range(EPOCHS):
    # Set both G&D train modes.
    G.train()
    D.train()

    # For logging in tensorboard
    loss_G_total, loss_D_total = 0., 0.
    loss_G_sub_total, loss_D_sub_total = 0., 0.

    for batch_idx, (real_A, real_B) in enumerate(train_loader):
        t1 = time.time()
        real_A = real_A.cuda()
        real_B = real_B.cuda()

        # ================= Update D ================== # 
        fake_B = G(real_A)

        # discriminator loss 
        disc_real = D(torch.cat((real_A, real_B), dim=1))
        disc_fake = D(torch.cat((real_A, fake_B.detach()), dim=1))
                              
        real_label = torch.ones_like(disc_real).cuda()
        fake_label = torch.zeros_like(disc_fake).cuda()
        loss_disc_real = BCE_criterion(disc_real, real_label)
        loss_disc_fake = BCE_criterion(disc_fake, fake_label)
        
        loss_D = loss_disc_real + loss_disc_fake
        
        # back prop.
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # ================= Update G ================== # 
        # generator loss
        disc_fake = D(torch.cat((real_A, fake_B), dim=1))

        real_label = torch.ones_like(disc_fake).cuda()
        loss_disc_fake = BCE_criterion(disc_fake, real_label)
        loss_L1 = nn.L1Loss()(fake_B, real_B)
        loss_G = loss_disc_fake + LAMBDA_L1 * loss_L1

        # back prop.
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        loss_D_total += loss_D.item()
        loss_G_total += loss_G.item()
        
        loss_G_sub_total += loss_G.item()
        loss_D_sub_total += loss_D.item()

        # print current states
        print_freq = 10
        if batch_idx % print_freq  == 0 and batch_idx>0:
            print('Epoch : {} || {}/{} || loss_G={:.3f} loss_D={:.3f} time={:.3f} secs/iter'.format(
                epoch, batch_idx, len(train_loader), 
                loss_G_sub_total/print_freq, loss_D_sub_total/print_freq, 
                time.time()-t1
            ))
            # ================= Genearte example samples ================== # 
            for bi, (real_A_test, real_B_test) in enumerate(test_loader):
                if bi>0:
                    break
                real_A_test = real_A_test[:NUM_EXAMPLES].cuda()
                real_B_test = real_B_test[:NUM_EXAMPLES].cuda()
                with torch.no_grad():
                    fake_B_test = G(real_A_test)
                    fake_B_test = fake_B_test[:NUM_EXAMPLES]

                real_A_samples = (real_A_test + 1)/2
                real_B_samples = (real_B_test + 1)/2
                fake_B_samples = (fake_B_test + 1)/2
                examples = torch.cat((real_A_samples, fake_B_samples, real_B_samples), dim=0)
                examples = make_grid(examples, nrow=NUM_EXAMPLES)
                
                # log images
                logger.add_image('Generated_pairs', examples, iterations)

            iterations += 1
            loss_G_sub_total = 0
            loss_D_sub_total = 0

    loss_G_total /= len(train_loader)
    loss_D_total /= len(train_loader)
    
    # logging on tensorboard
    logger.add_scalar('loss_G', loss_G_total, epoch)
    logger.add_scalar('loss_D', loss_D_total, epoch)

    # print current states
    print('Epoch : {} has done. AVG loss : loss_G={:.3f} loss_D={:.3f}'.format(
        epoch, loss_G_total, loss_D_total
    ))

     |████████████████████████████████| 317kB 2.8MB/s 


RuntimeError: ignored

In [ ]:
# Check Tensorboard.
%ls runs
%load_ext tensorboard
%tensorboard --logdir runs --port 9999